# Local to Wearehouse

Carregar os dados extraídos localmente utilizando o script get_json() utilizando como parâmetros:
- query: iterando listas.py
- cep: 88010560
- local: BR

Este script retorna o JSON bruto e extrai as variáveis de interesse.

Para futuras implementações, é recomendado extrair o JSON bruto e armazená-lo em um bucket S3.

Aqui será feita uma curadoria dos dados brutos para que sejam refinados para sua forma final, pronta pra ser consultada e analisada.

Os dados vêm de uma API bem estruturada, portanto esse processo será fácil.

### Descrição dos dados

| Nome da Coluna  | Nome Original | Tipo de Dados | Descrição                                                |
|-----------------|---------------|---------------|----------------------------------------------------------|
| data            | date          | datetime64[ns]| Data do registro da coleta                               |
| aisle_name      | aisle_name    | object        | Nome do corredor na loja                                 |
| product_name    | product_name  | object        | Nome do produto                                          |
| marca           | brand         | object        | Marca do produto                                         |
| preço           | price         | float64       | Preço do produto em R$                                   |
| pacote          | package       | object        | Informações da embalagem do produto (ml/kg/pacotes/etc)  |
| nome_loja       | store_name    | object        | Nome da loja (mercado, farmácia, etc)                    |
| cidade_loja     | store_city    | object        | Cidade onde a loja está localizada                       |
| termo_pesquisa  | search_term   | object        | Termo de pesquisa usado para encontrar o produto pela API|


### Carregando múltiplos arquivos json e convertendo em um dataframe

In [19]:
import pandas as pd
from src.get_json import get_and_create
from src.data import *

data_dir = 'data/'
df = read_json_files_as_dataframe(data_dir)

### Overview

In [22]:
df.shape

(363807, 9)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363807 entries, 0 to 363806
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   date          363807 non-null  datetime64[ns]
 1   aisle_name    363807 non-null  object        
 2   product_name  363807 non-null  object        
 3   brand         352694 non-null  object        
 4   price         363807 non-null  float64       
 5   package       363723 non-null  object        
 6   store_name    363807 non-null  object        
 7   store_city    363807 non-null  object        
 8   search_term   363807 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 25.0+ MB


### Removendo itens duplicados

In [28]:
# view the first 10 rows
df.head(10)

,date,aisle_name,product_name,brand,price,package,store_name,store_city,search_term
0,2023-08-06,Frutas frescas,Abacate,None,6.41,"Preço por kg, unidade: 500g aprox.",BIG by Carrefour,FLN,Abacate
1,2023-08-06,Frutas frescas,Avocado,None,19.19,"Preço por kg, unidade: 170g aprox.",BIG by Carrefour,FLN,Abacate
2,2023-08-06,Shampoo e condicionador,Kit de shampoo e condicionador abacate nutritivo,Dabelle,19.99,250ml+200ml,BIG by Carrefour,FLN,Abacate
3,2023-08-06,Shampoo e condicionador,Condicionador hidratação e nutrição óleo de co...,Suave,10.69,325ml,BIG by Carrefour,FLN,Abacate
4,2023-08-06,Shampoo e condicionador,Shampoo abacate e karité Joias da Natureza,Seda,13.89,325ml,BIG by Carrefour,FLN,Abacate
5,2023-08-06,Shampoo e condicionador,Shampoo abacate nutritivo,Dabelle,10.29,250ml,BIG by Carrefour,FLN,Abacate
6,2023-08-06,Produtos para finalização,Creme para pentear abacate nutritivo,Dabelle,12.99,270g,BIG by Carrefour,FLN,Abacate
7,2023-08-06,Produtos para finalização,Creme para pentear Joias da Natureza bomba aba...,Seda,24.19,700ml,BIG by Carrefour,FLN,Abacate
8,2023-08-06,Cremes e tratamentos corporais,Loção hidratante Óleos Essenciais flor de lara...,Nivea,18.99,400ml,BIG by Carrefour,FLN,Abacate
9,2023-08-06,Cremes e tratamentos corporais,Loção desodorante hidratante para as mãos com ...,Dove,19.69,75ml,BIG by Carrefour,FLN,Abacate


In [29]:
# Identify the duplicated rows
df[df.duplicated(keep=False)]

,date,aisle_name,product_name,brand,price,package,store_name,store_city,search_term
19118,2023-08-06,Higiene bucal,Antisséptico bucal ice proteção 360º fresh,Close Up,16.39,500ml,Atacadão,FLN,Antisséptico Bucal
19120,2023-08-06,Higiene bucal,Antisséptico bucal ice proteção 360º fresh,Close Up,16.39,500ml,Atacadão,FLN,Antisséptico Bucal
19254,2023-09-06,Oral Care,Antisséptico bucal ice proteção 360º fresh,Close Up,16.39,500ml,Atacadão,FLN,Antisséptico Bucal
19256,2023-09-06,Oral Care,Antisséptico bucal ice proteção 360º fresh,Close Up,16.39,500ml,Atacadão,FLN,Antisséptico Bucal
19389,2023-11-06,Higiene bucal,Antisséptico bucal ice proteção 360º fresh,Close Up,16.39,500ml,Atacadão,FLN,Antisséptico Bucal
...,...,...,...,...,...,...,...,...,...
362784,2023-06-21,Cuidados olhos e ouvidos,Gel otológico Auritop cães e gatos,Ourofino,106.50,30g,Cobasi,FLN,Água de Passar
362785,2023-06-21,Cuidados e manutenção,Refil para bebedouro fonte,Furacão Pet,17.90,1 unidade,Cobasi,FLN,Água de Passar
362786,2023-06-21,Açúcar e Adoçantes,Açúcar refinado,União,7.80,1kg,Kalunga,FLN,Água de Passar
362787,2023-06-21,Sistema de pastas,Pasta polionda média azul,Polibrás,6.40,"33,5x24,5x3,5cm",Kalunga,FLN,Água de Passar


Pode-se ver observações com todas as colunas com valores iguais. Se uma linha for igual a uma ou mais linhas, apenas a última linha a ser regitrada será mantida.

In [84]:
def drop_duplicates_keep_last(df):
    original_rows = df.shape[0]
    df_deduplicated = df.drop_duplicates(keep='last')
    removed_rows = original_rows - df_deduplicated.shape[0]
    print(f"Removed {removed_rows} row(s) from the original data.")
    return df_deduplicated

df_clean = drop_duplicates_keep_last(df)

Removed 7679 row(s) from the original data.


In [85]:
df_clean.shape

(356128, 9)

### Lidando com valores omissos

Não vamos imputar apenas identificar os valores omissos e substitui-las por uma string chamada de 'None'.

In [87]:
df_clean.isna().sum().sort_values(ascending=False).head(5)

brand           10726
package            84
date                0
aisle_name          0
product_name        0
dtype: int64

#### Brand (Marca)

In [108]:
df_clean[df_clean['brand'].isna()].sample(5)

,date,aisle_name,product_name,brand,price,package,store_name,store_city,search_term
85696,2023-06-21,Ovos de Páscoa,Gold bunny ao leite,None,37.90,100g,Lindt,FLN,Coelho
126499,2023-08-06,Higiene e cuidados pessoais,Esponja de banho amarela,None,6.99,1 unidade,Panvel,FLN,Esponja
247282,2023-06-16,Limpeza rosto,Sabonete facial esfoliante cade,None,179.90,150ml,L'Occitane en Provence,FLN,Máscara Facial
344596,2023-11-06,Frutas frescas,Mexerica uruguaia,None,29.69,"Preço por kg, unidade: 100g aprox.",Carrefour Hiper,FLN,Tangerina
40189,2023-09-06,Facial,Blush terracota,None,19.99,"3,5g",Panvel,FLN,Blush


In [88]:
df_clean[df_clean['brand'].isna()].head(5)

,date,aisle_name,product_name,brand,price,package,store_name,store_city,search_term
0,2023-08-06,Frutas frescas,Abacate,None,6.41,"Preço por kg, unidade: 500g aprox.",BIG by Carrefour,FLN,Abacate
1,2023-08-06,Frutas frescas,Avocado,None,19.19,"Preço por kg, unidade: 170g aprox.",BIG by Carrefour,FLN,Abacate
14,2023-08-06,Frutas frescas,Abacate,None,4.19,"Preço por kg, unidade: 500g aprox.",Carrefour Hiper,FLN,Abacate
15,2023-08-06,Frutas frescas,Avocado,None,19.19,"Preço por kg, unidade: 170g aprox.",Carrefour Hiper,FLN,Abacate
28,2023-08-06,Frutas frescas,Abacate,None,5.99,"Preço por kg, unidade: 600g aprox.",Atacadão,FLN,Abacate


**Justificativas**

- Tem verduras ou frutas, que geralmente não possuem uma marca.
- Produtos como pães que são produzidos pelo próprio mercado.

#### Package (Pacote)
Essa coluna se refera unidade de medida.
As unidades "ml", "kg", "número de pacotes" e "unidade" pertencem à categoria de medidas para representar quantidades, pesos e dimensionamentos de produtos.

- **A coluna `product_name` pode conter essa informação**

In [91]:
df_clean[df_clean['package'].isna()].sample(5)

,date,aisle_name,product_name,brand,price,package,store_name,store_city,search_term
231193,2023-06-19,Molhos,Molho de tomate refogado manjericão,Bonatelli,1.40,None,Atacadão,FLN,Manjericão
42805,2023-09-06,Treats,Osso Xisdog Bijuzinho Recheio,None,17.43,None,Cobasi,FLN,Bolacha Recheada
359219,2023-06-16,Utensílios cozinha,Kit de pá e escova Noviça,Betannin,35.99,None,Angeloni Supermercados,FLN,Vassoura
219811,2023-06-16,Unhas,Lixa de Unha,Ricca,3.99,None,Panvel,FLN,Lixa de Unha
2300,2023-06-19,Sistema digestivo e metabolismo,Epocler Flaconete Abacaxi 10ml,Hypermarcas,3.90,None,Pague Menos,FLN,Abacaxi


**Justificativas**

- A informação está contida na coluna `product_name`
- Tem verduras ou frutas, que geralmente não possuem uma marca.
- Produtos como pães que são produzidos pelo próprio mercado.
- Ter uma única unidade
- Conter explicitamente uma única unidade 

#### Substituindo valores omissos pela string 'None'

In [109]:
# Substituir omissos por string 'None'
df_clean = df_clean.fillna('None')

In [110]:
df_clean.isna().sum()

date            0
aisle_name      0
product_name    0
brand           0
price           0
package         0
store_name      0
store_city      0
search_term     0
dtype: int64

## Send data to bigquery


In [202]:
from google.api_core.exceptions import NotFound
from typing import List
from google.cloud import bigquery
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'cornershop-390320-9ae2464f7fb6.json'# Set your dataset_id to the ID of the dataset to create.

In [206]:
def create_dataset(client: bigquery.Client, dataset_name: str, location: str = "southamerica-east1") -> bigquery.Dataset:
    """
    Creates a new Google BigQuery dataset.

    Parameters
    ----------
    client : bigquery.Client
        The BigQuery client.
    dataset_name : str
        The name of the dataset to create.
    location : str
        The geographic location where the dataset should reside.

    Returns
    -------
    bigquery.Dataset
        The created dataset.

    Raises
    ------
    google.api_core.exceptions.Conflict
        If the Dataset already exists within the project.
    """
    # Construct full dataset ID
    dataset_id = f"{client.project}.{dataset_name}"

    # Check if dataset already exists
    try:
        client.get_dataset(dataset_id)  # Make an API request.
        print(f"Dataset {dataset_id} already exists.")
        return
    except NotFound:
        # Dataset does not exist
        pass

    # Construct a full Dataset object
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = location

    # Create the new Dataset
    dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
    print(f"Created dataset {client.project}.{dataset.dataset_id}")

    #return dataset.dataset_id 

def create_table(client: bigquery.Client, dataset_id: str, table_id: str, schema: List[bigquery.SchemaField]) -> bigquery.Table:
    """
    Creates a new Google BigQuery table.

    Parameters
    ----------
    client : bigquery.Client
        The BigQuery client.
    dataset_id : str
        The ID of the dataset in which to create the table.
    table_id : str
        The ID of the table to create.
    schema : List[bigquery.SchemaField]
        The schema of the table.

    Returns
    -------
    bigquery.Table
        The created table.

    Raises
    ------
    google.api_core.exceptions.Conflict
        If the Table already exists within the project.
    """
    
    # Construct full table ID
    full_table_id = f"{client.project}.{dataset_id}.{table_id}"

    # Check if table already exists
    try:
        client.get_table(full_table_id)  # Make an API request.
        print(f"Table {full_table_id} already exists.")
        return
    except NotFound:
        # Table does not exist
        pass

    # Construct a full Table object
    table = bigquery.Table(full_table_id, schema=schema)

    # Create the new Table
    table = client.create_table(table)  # Make an API request.
    print(f"Created table {table.full_table_id}")

# Define your schema
schema = [
    bigquery.SchemaField("date", "TIMESTAMP"),
    bigquery.SchemaField("aisle_name", "STRING"),
    bigquery.SchemaField("product_name", "STRING"),
    bigquery.SchemaField("brand", "STRING"),
    bigquery.SchemaField("price", "FLOAT64"),
    bigquery.SchemaField("package", "STRING"),
    bigquery.SchemaField("store_name", "STRING"),
    bigquery.SchemaField("store_city", "STRING"),
    bigquery.SchemaField("search_term", "STRING"),
]

# Create an instance of the BigQuery client
client = bigquery.Client()

dataset_id = 'local_silver'
table_id = 'cornershop_local_silver'
# Call the create_dataset function
create_dataset(client, dataset_id)
# Call the create_table function
create_table(client, dataset_id, table_id, schema)

Dataset cornershop-390320.local_silver already exists.
Table cornershop-390320.local_silver.cornershop_local_silver already exists.


In [215]:
from typing import List

def insert_data_to_table(client: bigquery.Client, dataset_id: str, table_id: str, data: List[dict], chunk_size: int = 1000):
    """
    Inserts data into a BigQuery table in smaller chunks.

    Parameters
    ----------
    client : bigquery.Client
        The BigQuery client.
    dataset_id : str
        The ID of the dataset containing the table.
    table_id : str
        The ID of the table to insert data into.
    data : List[dict]
        The data to be inserted. Each dictionary represents a row of data, with keys matching the column names.
    chunk_size : int, optional
        The size of each chunk for iterative insertion. Defaults to 1000.

    Returns
    -------
    bool
        True if the data was inserted successfully, False if there were any errors.
    """
    # Get the table reference
    table_ref = client.dataset(dataset_id).table(table_id)

    # Get the table object
    table = client.get_table(table_ref)

    # Split data into smaller chunks
    chunked_data = [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]

    # Insert data in smaller chunks
    for chunk in chunked_data:
        errors = client.insert_rows(table, chunk)

        if errors:
            print("Errors encountered while inserting data:")
            for error in errors:
                print(error)
            return False

    print("Data inserted successfully.")
    return True

# Convert the DataFrame to a list of dictionaries for insert_rows
rows_to_insert = df_clean.to_dict('records')

insert_data_to_table(client, dataset_id, table_id, rows_to_insert)                 

Data inserted successfully.


True

In [213]:
df_clean.shape

(356128, 9)

In [207]:
df_clean.to_parquet('cornershop_local_silver_220623.parquet')

## TODO

- `search_term` apresenta informações não associadas aos valores de `product_name`.
- `product_name` apresenta informações de quantidade associadas a `package`.

Temos que usar `package`, `product_name` para gerar identificar as quantidades apropriadas.

### product_name

In [119]:
# Identificar as strings que contêm números na coluna 'product_name'
df_clean[df_clean['product_name'].str.contains('\d')].head(2)

,date,aisle_name,product_name,brand,price,package,store_name,store_city,search_term
12,2023-08-06,"Iogurte, smoothies e kefir",Iogurte carolina 500g abacaxi e coco,Carolina,9.19,500g,BIG by Carrefour,FLN,Abacate
26,2023-08-06,"Iogurte, smoothies e kefir",Iogurte carolina 500g abacaxi e coco,Carolina,9.19,500g,Carrefour Hiper,FLN,Abacate


In [161]:
df_clean.query('package=="N"')[df_clean['product_name'].str.contains('\d')].sample(5)

C:\Users\Eric\AppData\Local\Temp\ipykernel_10072\3221363970.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_clean.query('package=="None"')[df_clean['product_name'].str.contains('\d')].sample(5)


,date,aisle_name,product_name,brand,price,package,store_name,store_city,search_term
165793,2023-06-14,"Curativos, gazes e bandagens",Porta-Cotonete Trama A1,Plasútil,6.49,None,Carrefour Hiper,FLN,Hastes Flexíveis
1808,2023-06-13,Sistema digestivo e metabolismo,Epocler Flaconete Abacaxi 10ml,Hypermarcas,3.90,None,Pague Menos,FLN,Abacaxi
165878,2023-06-15,"Curativos, gazes e bandagens",Porta-Cotonete Trama A1,Plasútil,6.49,None,Carrefour Hiper,FLN,Hastes Flexíveis
94074,2023-06-14,"Curativos, gazes e bandagens",Porta-Cotonete Trama A1,Plasútil,6.49,None,Carrefour Hiper,FLN,Cotonete
2043,2023-06-15,Sistema digestivo e metabolismo,Epocler Flaconete Abacaxi 10ml,Hypermarcas,3.90,None,Pague Menos,FLN,Abacaxi


In [189]:
import pandas as pd
import re

# Exemplo de dataframe
df = pd.DataFrame({'Produto': [
    'Abacaxi desidratado - 100g',
    'Iogurte carolina 500g abacaxi e coco',
    'Alivium 100mg gotas',
    'Epocler Flaconete Abacaxi 10ml',
    'Epocler Flaconete Abacaxi',
]})

# Função para extrair números seguidos por uma unidade
def extract_numbers(s):
    # A expressão regular abaixo significa "procurar um ou mais dígitos (\d+), 
    # seguidos opcionalmente por um ponto (\.?), seguidos por um ou mais dígitos (\d*), 
    # seguidos por uma ou mais letras que não são seguidas por um espaço ([a-zA-Z]+(?=\s))"
    matches = re.findall(r'\b\d+\.?\d*[a-zA-Z]+(?=\s|$)', s)
    # Se encontramos uma correspondência, retornamos a primeira. Caso contrário, retornamos None.
    return matches[0].strip() if matches else None

# Aplicar a função ao dataframe
df['Produto'].apply(extract_numbers)


0                         100g
1                         500g
2                        100mg
3                         10ml
4    Epocler Flaconete Abacaxi
Name: Produto, dtype: object

In [193]:
# Função para extrair números seguidos por uma unidade se coluna 'package' == 'None'
def extract_numbers(row):
    if row['package'] == 'None':
        matches = re.findall(r'\b\d+\.?\d*[a-zA-Z]+(?=\s|$)', row['product_name'])
        return matches[0].strip() if matches else None
    else:
        return None

extracted = df_clean.apply(extract_numbers, axis=1)    
    

In [197]:
extracted = extracted.fillna(df_clean['package'])

changes = extracted != df_clean['package']

changes.sum()

12

### Package 

In [198]:
df_clean.package.nunique()

1842

#### Package com string sem digitos

In [150]:
# Identificar as strings sem dígitos na coluna 'package'
non_numeric_strings = df_clean[df_clean['package'].str.match(r'^\D*$')]
non_numeric_strings.sample(10)

,date,aisle_name,product_name,brand,price,package,store_name,store_city,search_term
169355,2023-12-06,Mãos e pés,Creme de mãos mandacaru,L'Occitane Au Brésil,30.05,Embalagem,L'Occitane au Brésil,FLN,Hidratante para Pés
331125,2023-06-16,Peixes,Filé de salmão com pele congelado,Costa Sul,142.90,A granel,Angeloni Supermercados,FLN,Salmão
271471,2023-06-14,Frutas frescas,Pera red importada,None,3.06,A granel,Carrefour Hiper,FLN,Pera
8651,2023-06-13,Carne bovina,Alcatra,None,51.99,A granel,Carrefour Hiper,FLN,Alcatra
313080,2023-12-06,Rodenticidas e armadilhas,Cola pega rato,Krodec,7.99,Unidade,AmoresPet,FLN,Repelente
141204,2023-06-21,Peixes,Filé de salmão com pele congelado,Costa Sul,142.90,A granel,Angeloni Supermercados,FLN,Filé de Salmão
147370,2023-09-06,Chicken,Coxa com sobrecoxa de frango congelada,None,7.98,"A granel, Preço por kg",Carrefour Hiper,FLN,Frango Desossado
31455,2023-06-15,Carne de porco,Panceta suína congelada,Sulita,12.45,A granel,Atacadão,FLN,Barriga de Porco
181794,2023-06-14,Utensílios limpeza,Luva multiuso laranja plus,Sanro,7.59,Tam P,Atacadão,FLN,Laranja
354646,2023-06-21,"Sementes, farinhas e chás",Amendoim torrado sem sal - 100g,None,3.29,A granel,IMBUR Conceito Natural,FLN,Torrada


In [147]:
non_numeric_strings.package.unique()

array(['A granel', 'None', 'Unidade', 'unidade', 'Bivolt', 'unitário',
       'Embalagem', '-', 'Tam. GG', 'Tam. M', 'Tam. PP', 'Tam. G',
       'Tam. P', 'Tam G', 'Tam P', 'g', 'Pacote', 'Pote', 'Tamanho G',
       'A granel, Preço por kg', 'Caixa com duas coxinhas', 'a granel',
       'Tam. Único', 'Un', '.', 'Tamanho M', 'Granel, precio por kg',
       'Tam M', 'Tamanho GG'], dtype=object)